In [1]:
import os
os.chdir('../')

In [4]:
from Emotion_Detector.utils import *
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from Emotion_Detector.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    # Set memory growth to False to allocate the entire GPU memory
    tf.config.experimental.set_memory_growth(physical_devices[0], False)

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PretrainedModelTrainingConfig:
    root_dir: Path
    checkpoints_callback_dir : Path
    tensorboard_log_dir : Path
    model_history_dir : Path
    model_dir : Path
    model_file_path : str
    checkpoints_file_path : str
    model_history_file_path : Path
    model_accuracy_plot_path : Path
    model_loss_plot_path : Path
    train_dir: Path
    val_dir: Path
    param_en_epoch: int
    param_batch_size: int
    param_include_top: bool
    param_en_dense_1: int
    param_en_dense_2: int
    all_params : dict

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_pretrained_model_training_config(self) -> PretrainedModelTrainingConfig:
        config = self.config.pretrained_model_training


        create_directories([config.root_dir])
        create_directories([config.checkpoints_callback_dir])
        create_directories([config.tensorboard_log_dir])
        create_directories([config.model_history_dir])
        create_directories([config.model_dir])

        model_training_config = PretrainedModelTrainingConfig(
            root_dir = Path(config.root_dir),
            checkpoints_callback_dir = Path(config.checkpoints_callback_dir),
            tensorboard_log_dir = Path(config.tensorboard_log_dir),
            model_history_dir = Path(config.model_history_dir),
            model_dir = Path(config.model_dir),
            model_file_path =  config.model_file_path,
            checkpoints_file_path = config.checkpoints_file_path,
            model_history_file_path = Path(config.model_history_file_path),
            model_accuracy_plot_path = Path(config.model_accuracy_plot_path),
            model_loss_plot_path = Path(config.model_loss_plot_path),
            train_dir = Path(config.train_dir),
            val_dir = Path(config.val_dir),
            param_en_epoch= self.params.EN_EPOCH,
            param_batch_size  = self.params.EN_BATCH_SIZE,
            param_include_top = self.params.INCLUDE_TOP,
            param_en_dense_1 = self.params.EN_DENSE_1,
            param_en_dense_2 = self.params.EN_DENSE_2,
            all_params = self.params
        )
        return model_training_config

In [14]:
class Pretrained_Model_Training:
    def __init__(self, config: PretrainedModelTrainingConfig):
        self.config = config

    def _get_data(self):
        train_data, val_data = get_augmented_data(train_dir_path= self.config.train_dir,
                                                  val_dir_path= self.config.val_dir,
                                                  params= self.config.all_params
                                                  )
        return train_data, val_data
        
    def _get_model(self):
        backbone = tf.keras.applications.efficientnet.EfficientNetB4(
            include_top= self.config.param_include_top,
            weights='imagenet',
            input_shape=(self.config.all_params['IMAGE_SIZE'], self.config.all_params['IMAGE_SIZE'], 3)
        )
        backbone.trainable = False
        model = tf.keras.Sequential([
            Input(shape = (self.config.all_params['IMAGE_SIZE'], self.config.all_params['IMAGE_SIZE'], 3)),
            backbone,
            GlobalAveragePooling2D(),
            Dense(self.config.param_en_dense_1, activation = "relu"),
            BatchNormalization(),
            Dense(self.config.param_en_dense_2, activation = "relu"),
            Dense(self.config.all_params["NUM_CLASSES"], activation = "softmax"),
        ])

        return model
    
    def train_model(self):
        
        logger.info(f"Reconstructing the data from tensorflow records...")
        train_data, val_data = self._get_data()
        logger.info(f"Data loaded successfully.")

        logger.info(f"Geting the Pretrained Model...")
        model = self._get_model()
        logger.info(f"Model Loaded Successfully...")

        logger.info(f"Preparing Checkpoint Callback...")
        checkpoint_callback = ModelCheckpoint(
            filepath= self.config.checkpoints_file_path,
            save_weights_only=True,
            save_best_only=True,
            monitor='val_loss',
            verbose=1
        )

        logger.info(f"Preparing Tensorboard Callback...")
        tensorboard_callback = TensorBoard(log_dir=self.config.tensorboard_log_dir, histogram_freq=1)


        loss_function = CategoricalCrossentropy()
        metrics = [CategoricalAccuracy(name = "accuracy")]

        logger.info(f"Compiling the model...")
        model.compile(
                optimizer = Adam(learning_rate= 0.001),
                loss = loss_function,
                metrics = metrics
            )
        
        logger.info(f"Starting training with {self.config.param_en_epoch} epochs and validation data...")
        history = model.fit(train_data,batch_size = self.config.param_batch_size, epochs = self.config.param_en_epoch,validation_data = val_data, callbacks = [checkpoint_callback, tensorboard_callback], verbose = 1)
        logger.info(f"Model Training Completed Successfully.")
        logger.info(f"Model Summary /n {model.summary()}")

        logger.info(f"Saving the model at {self.config.model_dir}...")
        model.save(self.config.model_dir, save_format = 'tf')
        logger.info(f"Model successfully saved at {self.config.model_dir}.")

        logger.info(f"Saving the model history at {self.config.model_history_file_path}...")
        save_json(path = self.config.model_history_file_path, data = history.history)
        logger.info(f"Model history successfully saved at {self.config.model_history_file_path}.")

        logger.info(f"Saving the model accuracy plot at {self.config.model_accuracy_plot_path}...")
        save_plt_fig(x = history.history['accuracy'],
                     y = history.history['val_accuracy'],
                     title = "Model Accuracy",
                     xlabel ='Epochs',
                     ylabel = "Accuracy",
                     legends= ['Train', 'Validation'],
                     fig_path = self.config.model_accuracy_plot_path)
        
        logger.info(f"Saving the model loss plot at {self.config.model_loss_plot_path}...")
        save_plt_fig(x = history.history['loss'],
                     y = history.history['val_loss'],
                     title = "Model loss",
                     xlabel ='Epochs',
                     ylabel = "Loss",
                     legends= ['Train', 'Validation'],
                     fig_path = self.config.model_loss_plot_path)
        logger.info(f"Model results saved successfully.")

In [15]:
try:
    config = ConfigurationManager()
    pretrained_model_training_config = config.get_pretrained_model_training_config()
    model_trainer = Pretrained_Model_Training(config = pretrained_model_training_config)
    model_trainer.train_model() 
except Exception as e:
    raise e

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.
Epoch 1/60
 41/898 [>.............................] - ETA: 8:40 - loss: 2.1054 - accuracy: 0.3468

KeyboardInterrupt: 

In [ ]:
from Emotion_Detector.utils import reconstruct_data_from_tfrecords

In [ ]:
import os
os.chdir('../')

In [ ]:
path = 'artifacts/data_preprocessing/train_tfrecords/shard_{:02d}.tfrecord'
num_shards = 10
batch_size = 32

In [ ]:
parsed_data = reconstruct_data_from_tfrecords(path, num_shards, batch_size )